In [1]:
from inception_resnet_v2 import InceptionResNetV2

Using TensorFlow backend.
/home/insideout/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
PATH = "/path/to/train/and/valid/dir"
sz=299
batch_size=64

folder hierarchy

root_folder
|_ train
|_ valid


In [6]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense
from keras.optimizers import Adam, RMSprop, Nadam
#from keras.applications import ResNet50
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization
from keras import backend as K

In [7]:
train_data_dir = f'{PATH}train'
validation_data_dir = f'{PATH}valid'
batch_size = 64

In [8]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
    target_size=(sz, sz),
    batch_size=batch_size, class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
    shuffle=False,
    target_size=(sz, sz),
    batch_size=batch_size, class_mode='categorical')

Found 16262 images belonging to 120 classes.
Found 4122 images belonging to 120 classes.


In [9]:
base_model= InceptionResNetV2()

In [10]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [11]:
base_model.layers.pop()


In [12]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

                                                                 activation_100[0][0]             
__________________________________________________________________________________________________
block17_6_conv (Conv2D)         (None, None, None, 1 418880      block17_6_mixed[0][0]            
__________________________________________________________________________________________________
block17_6 (Lambda)              (None, None, None, 1 0           block17_5_ac[0][0]               
                                                                 block17_6_conv[0][0]             
__________________________________________________________________________________________________
block17_6_ac (Activation)       (None, None, None, 1 0           block17_6[0][0]                  
__________________________________________________________________________________________________
conv2d_102 (Conv2D)             (None, None, None, 1 139264      block17_6_ac[0][0]               
__________

In [13]:
#base_model.layers.pop()

In [14]:
inp=base_model.input
output = base_model.layers[-1].output

In [15]:
x = output
#x = GlobalAveragePooling2D()(x)
#x= BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(x)
#x = Dropout(0.5, noise_shape=None, seed=None)(x)
#x = Dense(1024, activation='relu')(x)
#x= BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(x)
#x = Dropout(0.5, noise_shape=None, seed=None)(x)
#x = Dense(512, activation='relu')(x)
predictions = Dense(120, activation='softmax')(x)

In [16]:
model = Model(inputs=inp, outputs=predictions)
optimizer = Adam(lr=0.001)
for layer in base_model.layers: layer.trainable = False
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [ ]:
lr_reducer = ReduceLROnPlateau(monitor='val_acc', factor=np.sqrt(0.1),
                                    cooldown=0, patience=5, min_lr=1e-5)
#model_checkpoint= ModelCheckpoint(weights_file, monitor="val_acc", save_best_only=True,save_weights_only=True, verbose=1)
callbacks=[lr_reducer]


In [ ]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=5, workers=4, callbacks=callbacks,
        validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/5
255/254 [==============================] - 269s 1s/step - loss: 1.3423 - acc: 0.7463 - val_loss: 0.4201 - val_acc: 0.8802
Epoch 2/5
255/254 [==============================] - 264s 1s/step - loss: 0.5307 - acc: 0.8487 - val_loss: 0.4050 - val_acc: 0.8751
Epoch 3/5
255/254 [==============================] - 260s 1s/step - loss: 0.4811 - acc: 0.8560 - val_loss: 0.4143 - val_acc: 0.8775
Epoch 4/5
255/254 [==============================] - 261s 1s/step - loss: 0.4487 - acc: 0.8626 - val_loss: 0.3976 - val_acc: 0.8804
Epoch 5/5
221/254 [=========================>....] - ETA: 26s - loss: 0.4240 - acc: 0.8664

In [ ]:
split_at = 140
optimizer = Adam(lr=1e-3)
for layer in model.layers[:split_at]: layer.trainable = False
for layer in model.layers[split_at:]: layer.trainable = True
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])  

In [ ]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=10, 
                    validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)